In [1]:
import pandas as pd

In [26]:
data = pd.read_excel('../dataset/data_new.xlsx')

In [27]:
df = pd.DataFrame(data)

In [28]:
df.head()

,story_id,story_title,sentence_id,word,pos_tag,entity_tag,head,deprel,word_id
0,0,anak_ririh,0,Pan,NNP,B-PNAME,2,comp,1.0
1,0,anak_ririh,0,Karsa,NNP,I-PNAME,6,subj,2.0
2,0,anak_ririh,0,ajaka,IN,O,4,case,3.0
3,0,anak_ririh,0,pianakne,NN,O,6,subj,4.0
4,0,anak_ririh,0,muani,NN,O,4,appos,5.0


In [29]:
word_concat = [[]]
pos_tag_concat = [[]]
head_concat = [[]]
deprel_concat = [[]]
sentence_id = 0

for index, row in df.iterrows():
    if row['pos_tag'] != 'Z':
        word_concat[sentence_id].append(row['word'])
        pos_tag_concat[sentence_id].append(row['pos_tag'])
        head_concat[sentence_id].append(int(row['head']))
        deprel_concat[sentence_id].append(row['deprel'])
    else:
        sentence_id += 1
        word_concat.append([])
        pos_tag_concat.append([])
        head_concat.append([])
        deprel_concat.append([])


In [30]:
df_concat = pd.DataFrame({
    'sentence' : word_concat,
    'pos_tag' : pos_tag_concat,
    'head' : head_concat,
    'deprel' : deprel_concat,
})

In [31]:
print(df[df['head'].isnull()])

Empty DataFrame
Columns: [story_id, story_title, sentence_id, word, pos_tag, entity_tag, head, deprel, word_id]
Index: []


In [199]:
df_concat.head()

,sentence,pos_tag,head,deprel
0,"[Pan, Karsa, ajaka, pianakne, muani, nanggap, ...","[NNP, NNP, IN, NN, NN, VB, NN, VB, NN, IN, NN,...","[2, 6, 4, 6, 4, 0, 6, 6, 8, 11, 12, 8, 12]","[comp, subj, case, subj, appos, root, obj, xco..."
1,"[Uli, semeng, maka, sanja, ia, ajaka, dadua, t...","[IN, NN, IN, NN, PR, IN, CD, NEG, NN, VB, SC, ...","[2, 10, 4, 10, 10, 7, 5, 9, 10, 0, 13, 12, 11]","[case, nmod, case, nmod, subj, case, nmod, adv..."
2,"[Kenehne, apang, gegaene, enggal, pragat, tur,...","[NN, SC, NN, JJ, JJ, CC, RB, VB, NN]","[5, 5, 5, 5, 0, 8, 8, 5, 8]","[subj, conj, obj, amod, root, conj, advmod, cc..."
3,"[Telung, dina, ia, magae, tan, parerenan, seme...","[CD, NN, PR, VB, NEG, VB, NN, RB, JJ, VB, JJ]","[2, 4, 4, 0, 6, 4, 4, 9, 7, 4, 10]","[nmod, nmod, subj, root, advmod, xcomp, obj, a..."
4,"[Kendelne, tara, bakat, ban, nuturang]","[NN, NEG, MD, RB, VB]","[5, 3, 5, 5, 0]","[obj, advmod, xcomp, advmod, root]"


In [200]:
def parse_data(sentence):
    array_dict = []

    for index_i, word_i in enumerate(sentence['sentence']):
        for index_j, word_j in enumerate(sentence['sentence']):
            if word_i != word_j:
                deps_score = 0
                if sentence['head'][index_i] == index_j+1:
                    deps_score = 1
                distance = abs(index_i - index_j)
                token_dict = {
                    'word_i' : word_i,
                    'word_j' : word_j,
                    'pos_i' : sentence['pos_tag'][index_i],
                    'pos_j' : sentence['pos_tag'][index_j],
                    'distance' : distance,
                    'deps_score' : deps_score
                }
                array_dict.append(token_dict)

    return array_dict

In [201]:
sents = []

for index, row in df_concat.iterrows():
    sents += parse_data(row)

In [202]:
df_deps = pd.DataFrame(sents)

In [205]:
df_deps['deps_score_min'] = 1 - df_deps['deps_score']

In [206]:
df_deps.head()

,word_i,word_j,pos_i,pos_j,distance,deps_score,deps_score_min
0,Pan,Karsa,NNP,NNP,1,1,0
1,Pan,ajaka,NNP,IN,2,0,1
2,Pan,pianakne,NNP,NN,3,0,1
3,Pan,muani,NNP,NN,4,0,1
4,Pan,nanggap,NNP,VB,5,0,1


In [208]:
df_deps.to_excel('../dataset/dataset_score.xlsx', index=False)

In [209]:
import pandas as pd

df_deps = pd.read_excel('../dataset/dataset_score.xlsx')

In [210]:
df_deps.head()

,word_i,word_j,pos_i,pos_j,distance,deps_score,deps_score_min
0,Pan,Karsa,NNP,NNP,1,1,0
1,Pan,ajaka,NNP,IN,2,0,1
2,Pan,pianakne,NNP,NN,3,0,1
3,Pan,muani,NNP,NN,4,0,1
4,Pan,nanggap,NNP,VB,5,0,1


In [211]:
import fasttext
emb_model = fasttext.load_model('../model_fasttext.bin')

def word_embedding(word):
    return emb_model.get_word_vector(word)

In [212]:
import numpy as np

word_concat = []

for index, row in df_deps.iterrows():
    word_concat.append(np.concatenate((word_embedding(row['word_i']), word_embedding(row['word_j']))))


In [213]:
df_deps['concat_emb'] = word_concat

In [214]:
df_deps.head()

,word_i,word_j,pos_i,pos_j,distance,deps_score,deps_score_min,concat_emb
0,Pan,Karsa,NNP,NNP,1,1,0,"[0.0022746837, 0.43677863, -0.30973542, -0.698..."
1,Pan,ajaka,NNP,IN,2,0,1,"[0.0022746837, 0.43677863, -0.30973542, -0.698..."
2,Pan,pianakne,NNP,NN,3,0,1,"[0.0022746837, 0.43677863, -0.30973542, -0.698..."
3,Pan,muani,NNP,NN,4,0,1,"[0.0022746837, 0.43677863, -0.30973542, -0.698..."
4,Pan,nanggap,NNP,VB,5,0,1,"[0.0022746837, 0.43677863, -0.30973542, -0.698..."


# Neural Network

In [113]:
len(df_deps['concat_emb'][0])

200

In [32]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.nn import MSELoss

In [41]:
class EdgeScorer(nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super(EdgeScorer, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)  # Hidden layer
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_dim, 1)         # Output layer
        self.sigmoid = nn.Sigmoid()                # Output: Probabilitas (0-1)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.sigmoid(x)
        return x


In [42]:
import random

data_to_feature = list(zip(list(df_deps['concat_emb']), list(df_deps['deps_score_min'])))
data_to_feature_dupl = data_to_feature*5
random.shuffle(data_to_feature_dupl)


NameError: name 'df_deps' is not defined

In [353]:
data1, data2 = zip(*data_to_feature_dupl)
X = torch.tensor(list(data1), dtype=torch.float32)
y = torch.tensor(list(data2), dtype=torch.float32)


In [354]:
# Parameter model
input_dim = 200  # Dimensi vektor input (embedding kata1 + kata2)
hidden_dim = 128  # Dimensi hidden layer

# Inisialisasi model, loss function, dan optimizer
model = EdgeScorer(input_dim, hidden_dim)
criterion = nn.BCELoss()  # Binary Cross-Entropy Loss
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
epochs = 100
for epoch in range(epochs):
    optimizer.zero_grad()
    outputs = model(X).squeeze()  # Output probabilitas
    loss = criterion(outputs, y)
    loss.backward()
    optimizer.step()

    # Hitung MAE
    mae = torch.mean(torch.abs(outputs - y))  # MAE per batch

    if (epoch + 1) % 10 == 0:
        print(f"Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}, MAE: {mae.item():.4f}")

torch.save(model.state_dict(), 'edge_scorer.pth')


Epoch [10/100], Loss: 0.3337, MAE: 0.2680
Epoch [20/100], Loss: 0.1952, MAE: 0.1127
Epoch [30/100], Loss: 0.1945, MAE: 0.0763
Epoch [40/100], Loss: 0.1966, MAE: 0.0733
Epoch [50/100], Loss: 0.1921, MAE: 0.0808
Epoch [60/100], Loss: 0.1903, MAE: 0.0908
Epoch [70/100], Loss: 0.1904, MAE: 0.0954
Epoch [80/100], Loss: 0.1902, MAE: 0.0941
Epoch [90/100], Loss: 0.1901, MAE: 0.0915
Epoch [100/100], Loss: 0.1900, MAE: 0.0905


In [34]:
def find_score(sentence, model):
    sentence_split = sentence.split()
    edge = []
    scores = []

    for index_i, word_i in enumerate(sentence_split):
        for index_j, word_j in enumerate(sentence_split):
            if index_i != index_j:
                emb_i = word_embedding(word_i)
                emb_j = word_embedding(word_j)
                input_vec = torch.tensor(np.concatenate((emb_i, emb_j)), dtype=torch.float32).unsqueeze(0)
                with torch.no_grad():
                    prob = model(input_vec).item()
                edge.append([word_i, word_j])
                scores.append(prob)
    return scores, edge



# MST Chu-Liu/Edmonds

In [307]:
import networkx as nx

sentence = 'Ditu lantas Pan Karsa ngantungan baju muah capilne ditongose ngantungan buka ane suba-suba'

In [316]:
def edge_score_map(model, sentence):
    input_x = []

    scores, edge = find_score(sentence, model)

    for i in range(len(scores)):
        input_x.append((edge[i][0], edge[i][1], scores[i]))

    return input_x

In [318]:
input_dim = 200
hidden_dim = 128
model = EdgeScorer(input_dim, hidden_dim)
model.load_state_dict(torch.load('edge_scorer.pth'))
model.eval()

input_x = edge_score_map(model, sentence)
input_x

C:\Users\widi arsa\AppData\Local\Temp\ipykernel_20040\3929718557.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('edge_scorer.pth'))


[('Ditu', 'lantas', 0.9576436877250671),
 ('Ditu', 'Pan', 0.9711195230484009),
 ('Ditu', 'Karsa', 0.8944690227508545),
 ('Ditu', 'ngantungan', 0.9580451846122742),
 ('Ditu', 'baju', 0.8870452046394348),
 ('Ditu', 'muah', 0.9630140066146851),
 ('Ditu', 'capilne', 0.9088645577430725),
 ('Ditu', 'ditongose', 0.9134529829025269),
 ('Ditu', 'ngantungan', 0.9580451846122742),
 ('Ditu', 'buka', 0.9504033923149109),
 ('Ditu', 'ane', 0.9738821387290955),
 ('Ditu', 'suba-suba', 0.9028505086898804),
 ('lantas', 'Ditu', 0.9546430706977844),
 ('lantas', 'Pan', 0.9798765778541565),
 ('lantas', 'Karsa', 0.9246781468391418),
 ('lantas', 'ngantungan', 0.970646858215332),
 ('lantas', 'baju', 0.9191937446594238),
 ('lantas', 'muah', 0.9741634130477905),
 ('lantas', 'capilne', 0.9352399110794067),
 ('lantas', 'ditongose', 0.9385899305343628),
 ('lantas', 'ngantungan', 0.970646858215332),
 ('lantas', 'buka', 0.9652169942855835),
 ('lantas', 'ane', 0.9818172454833984),
 ('lantas', 'suba-suba', 0.93083918094

In [281]:
import networkx as nx

# Data dalam bentuk edge dan bobot
def mst_parser(input_x):
    edges = input_x

    # Membuat graf terarah
    G = nx.DiGraph()
    for u, v, weight in edges:
        G.add_edge(u, v, weight=weight)

    # Mencari MST menggunakan Chu-Liu/Edmonds
    mst = nx.minimum_spanning_arborescence(G)

    # Menampilkan hasil MST
    # for u, v, weight in mst.edges(data=True):
    #     print(f'{v} -> {u} dengan bobot {weight["weight"]}')
    return mst


In [282]:
output = list(mst_parser(input_x).edges)
output

[('baju', 'buka'),
 ('baju', 'ane'),
 ('baju', 'suba-suba'),
 ('baju', 'Ditu'),
 ('baju', 'lantas'),
 ('baju', 'Pan'),
 ('baju', 'Karsa'),
 ('baju', 'ngantungan'),
 ('baju', 'capilne'),
 ('baju', 'ditongose'),
 ('muah', 'baju')]

In [335]:
def construct_head(input_x, sentence_split):
    edge_score = list(mst_parser(input_x).edges)
    
    tail = [x[1] for x in edge_score]
    dependence_word = []

    for index, value in enumerate(sentence_split):
        if value in tail:
            head = edge_score[int(tail.index(value))]
            dependence_word.append([value, head[0]])
        else:
            dependence_word.append([value])
            
    head_output = []

    for dependence in dependence_word:
        if len(dependence) == 1:
            head_output.append([dependence[0], 0])
        else:
            head_output.append([dependence[0], (sentence.split()).index(dependence[1])])
    return head_output


In [291]:
head_features = construct_head(input_x, sentence.split())
d

[['Ditu', 5],
 ['lantas', 5],
 ['Pan', 5],
 ['Karsa', 5],
 ['ngantungan', 5],
 ['baju', 6],
 ['muah', 0],
 ['capilne', 5],
 ['ditongose', 5],
 ['ngantungan', 5],
 ['buka', 5],
 ['ane', 5],
 ['suba-suba', 5]]

# Edge-Label

In [6]:
import pandas as pd

In [35]:
data = pd.read_excel('../dataset/data_new.xlsx')

In [36]:
df = pd.DataFrame(data)

In [37]:
df.head()

,story_id,story_title,sentence_id,word,pos_tag,entity_tag,head,deprel,word_id
0,0,anak_ririh,0,Pan,NNP,B-PNAME,2,comp,1.0
1,0,anak_ririh,0,Karsa,NNP,I-PNAME,6,subj,2.0
2,0,anak_ririh,0,ajaka,IN,O,4,case,3.0
3,0,anak_ririh,0,pianakne,NN,O,6,subj,4.0
4,0,anak_ririh,0,muani,NN,O,4,appos,5.0


In [38]:
raw_feature = [[]]

temp_index = 0

for index, value in df.iterrows():
    if index != len(df)-1 and df['sentence_id'][index] != df['sentence_id'][index+1]:
        raw_feature.append([])
        temp_index += 1
    # print(raw_feature)
    if value['word'] != '.':
        raw_feature[temp_index].append([value['word'], value['pos_tag'], value['head'], value['deprel']])


In [39]:
raw_feature[0]

[['Pan', 'NNP', 2, 'comp'],
 ['Karsa', 'NNP', 6, 'subj'],
 ['ajaka', 'IN', 4, 'case'],
 ['pianakne', 'NN', 6, 'subj'],
 ['muani', 'NN', 4, 'appos'],
 ['nanggap', 'VB', 0, 'root'],
 ['upah', 'NN', 6, 'obj'],
 ['ngae', 'VB', 6, 'xcomp'],
 ['semer', 'NN', 8, 'obj'],
 ['di', 'IN', 11, 'case'],
 ['sisin', 'NN', 12, 'nmod'],
 ['rurunge', 'NN', 8, 'nmod'],
 ['gede', 'JJ', 12, 'amod']]

In [10]:
import random

duplikat_raw_feature = raw_feature*5
random.shuffle(duplikat_raw_feature)
duplikat_raw_feature[0]

[['Wenten', 'VB', 3, 'comp'],
 ['kocap', 'VB', 3, 'comp'],
 ['katuturan', 'VB', 11, 'ccomp'],
 ['satua', 'NN', 11, 'subj'],
 ['saking', 'IN', 6, 'case'],
 ['jagat', 'NN', 11, 'nmod'],
 ['Jembrana', 'NNP', 6, 'appos'],
 ['sane', 'CC', 11, 'conj'],
 ['sampun', 'RB', 10, 'advmod'],
 ['lumbrah', 'JJ', 14, 'amod'],
 ['kabaosang', 'VB', 0, 'root'],
 ['antuk', 'IN', 13, 'case'],
 ['para', 'DT', 14, 'det'],
 ['janane', 'NN', 11, 'obj'],
 ['sane', 'CC', 18, 'conj'],
 ['seneng', 'JJ', 18, 'amod'],
 ['ka', 'IN', 18, 'case'],
 ['tajen', 'VB', 11, 'ccomp'],
 ['wenten', 'VB', 18, 'xcomp'],
 ['kabaos', 'VB', 18, 'xcomp'],
 ['ayam', 'NN', 20, 'obj'],
 ['ijo', 'JJ', 21, 'amod'],
 ['sane', 'CC', 24, 'conj'],
 ['mawasta', 'VB', 20, 'ccomp'],
 ['Ijo', 'NNP', 26, 'comp'],
 ['Sambu', 'NNP', 24, 'obj'],
 ['nanging', 'CC', 29, 'conj'],
 ['kawentenane', 'NN', 24, 'nmod'],
 ['arang', 'JJ', 28, 'amod'],
 ['pisan', 'RB', 29, 'advmod']]

In [11]:
train =  duplikat_raw_feature[:round(len(duplikat_raw_feature)*0.8)]
test = duplikat_raw_feature[round(len(duplikat_raw_feature)*0.8):]

In [360]:
test

[[['I', 'DT', 2, 'det'],
  ['Kerkuak', 'NNP', 4, 'subj'],
  ['lantas', 'RB', 4, 'advmod'],
  ['nunden', 'VB', 0, 'root'],
  ['I', 'DT', 6, 'det'],
  ['Angsa', 'NNP', 4, 'obj'],
  ['numanang', 'VB', 4, 'ccomp'],
  ['be', 'NN', 7, 'obj'],
  ['We', 'UH', 10, 'intj'],
  ['cai', 'PR', 14, 'subj'],
  ['Angsa', 'NNP', 10, 'appos'],
  ['yen', 'SC', 14, 'conj'],
  ['jani', 'RB', 14, 'nmod'],
  ['amah', 'VB', 7, 'xcomp'],
  ['bene', 'NN', 14, 'obj'],
  ['onyangan', 'JJ', 15, 'amod'],
  ['sinah', 'CC', 22, 'conj'],
  ['buin', 'RB', 19, 'advmod'],
  ['mani', 'RB', 20, 'advmod'],
  ['poan', 'RB', 21, 'advmod'],
  ['sing', 'NEG', 22, 'advmod'],
  ['ada', 'VB', 14, 'ccomp'],
  ['buin', 'RB', 24, 'advmod'],
  ['ane', 'CC', 26, 'conj'],
  ['lakar', 'MD', 26, 'advmod'],
  ['amah', 'VB', 22, 'xcomp'],
  ['tumanang', 'VB', 26, 'xcomp'],
  ['bene', 'NN', 27, 'obj'],
  ['masaut', 'VB', 27, 'ccomp'],
  ['lantas', 'RB', 31, 'advmod'],
  ['I', 'DT', 32, 'det'],
  ['Angsa', 'NNP', 29, 'subj'],
  ['ye', 'UH', 34

In [13]:
def word2features(sent, i):
    word = sent[i][0]
    postag = sent[i][1]
    features = [
        'bias',
        'word.lower=' + word.lower(),
        'word.isupper=%s' % word.isupper(),
        'word.istitle=%s' % word.istitle(),
        'postag=' + postag,
        'postag[:2]=' + postag[:2],
        'index=' + str(i),
        'head=' + str(sent[i][2])
    ]
    if i > 0:
        word1 = sent[i-1][0]
        postag1 = sent[i-1][1]
        features.extend([
            '-1:word.lower=' + word1.lower(),
            '-1:word.istitle=%s' % word1.istitle(),
            '-1:word.isupper=%s' % word1.isupper(),
            '-1:postag=' + postag1,
            '-1:postag[:2]=' + postag1[:2],
        ])
    else:
        features.append('BOS')
        
    if i < len(sent)-1:
        word1 = sent[i+1][0]
        postag1 = sent[i+1][1]
        features.extend([
            '+1:word.lower=' + word1.lower(),
            '+1:word.istitle=%s' % word1.istitle(),
            '+1:word.isupper=%s' % word1.isupper(),
            '+1:postag=' + postag1,
            '+1:postag[:2]=' + postag1[:2],
        ])
    else:
        features.append('EOS')
                
    return features


def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    return [label for token, postag, head, label in sent]


In [14]:
X_train = [sent2features(s) for s in train]
y_train = [sent2labels(s) for s in train]   

X_test = [sent2features(s) for s in test]
y_test = [sent2labels(s) for s in test]

In [359]:
X_test[0]

[['bias',
  'word.lower=i',
  'word.isupper=True',
  'word.istitle=True',
  'postag=DT',
  'postag[:2]=DT',
  'index=0',
  'head=2',
  'BOS',
  '+1:word.lower=kerkuak',
  '+1:word.istitle=True',
  '+1:word.isupper=False',
  '+1:postag=NNP',
  '+1:postag[:2]=NN'],
 ['bias',
  'word.lower=kerkuak',
  'word.isupper=False',
  'word.istitle=True',
  'postag=NNP',
  'postag[:2]=NN',
  'index=1',
  'head=4',
  '-1:word.lower=i',
  '-1:word.istitle=True',
  '-1:word.isupper=True',
  '-1:postag=DT',
  '-1:postag[:2]=DT',
  '+1:word.lower=lantas',
  '+1:word.istitle=False',
  '+1:word.isupper=False',
  '+1:postag=RB',
  '+1:postag[:2]=RB'],
 ['bias',
  'word.lower=lantas',
  'word.isupper=False',
  'word.istitle=False',
  'postag=RB',
  'postag[:2]=RB',
  'index=2',
  'head=4',
  '-1:word.lower=kerkuak',
  '-1:word.istitle=True',
  '-1:word.isupper=False',
  '-1:postag=NNP',
  '-1:postag[:2]=NN',
  '+1:word.lower=nunden',
  '+1:word.istitle=False',
  '+1:word.isupper=False',
  '+1:postag=VB',
  

In [253]:
X_train[0]

[['bias',
  'word.lower=makeneh',
  'word.isupper=False',
  'word.istitle=True',
  'postag=VB',
  'postag[:2]=VB',
  'index=0',
  'head=0',
  'BOS',
  '+1:word.lower=lantas',
  '+1:word.istitle=False',
  '+1:word.isupper=False',
  '+1:postag=RB',
  '+1:postag[:2]=RB'],
 ['bias',
  'word.lower=lantas',
  'word.isupper=False',
  'word.istitle=False',
  'postag=RB',
  'postag[:2]=RB',
  'index=1',
  'head=3',
  '-1:word.lower=makeneh',
  '-1:word.istitle=True',
  '-1:word.isupper=False',
  '-1:postag=VB',
  '-1:postag[:2]=VB',
  '+1:word.lower=i',
  '+1:word.istitle=True',
  '+1:word.isupper=True',
  '+1:postag=DT',
  '+1:postag[:2]=DT'],
 ['bias',
  'word.lower=i',
  'word.isupper=True',
  'word.istitle=True',
  'postag=DT',
  'postag[:2]=DT',
  'index=2',
  'head=4',
  '-1:word.lower=lantas',
  '-1:word.istitle=False',
  '-1:word.isupper=False',
  '-1:postag=RB',
  '-1:postag[:2]=RB',
  '+1:word.lower=angsa',
  '+1:word.istitle=True',
  '+1:word.isupper=False',
  '+1:postag=NNP',
  '+1:

In [251]:
import pycrfsuite

trainer = pycrfsuite.Trainer(verbose=False)

for xseq, yseq in zip(X_train, y_train):
    trainer.append(xseq, yseq)

In [255]:
trainer.set_params({
    'c1': 1.0,   # coefficient for L1 penalty
    'c2': 1e-3,  # coefficient for L2 penalty
    'max_iterations': 100,  # stop earlier

    # include transitions that are possible, but not observed
    'feature.possible_transitions': True
})
trainer.train('label_scorer.crfsuite')

In [256]:
%%time
trainer.train('label_scorer.crfsuite')

CPU times: total: 703 ms
Wall time: 1.06 s


In [3]:
from itertools import chain
import nltk
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import LabelBinarizer
import sklearn
import pycrfsuite

print(sklearn.__version__)

1.5.2


In [18]:
def bio_classification_report(y_true, y_pred):
    """
    Classification report for a list of BIO-encoded sequences.
    It computes token-level metrics and discards "O" labels.
    
    Note that it requires scikit-learn 0.15+ (or a version from github master)
    to calculate averages properly!
    """
    lb = LabelBinarizer()
    y_true_combined = lb.fit_transform(list(chain.from_iterable(y_true)))
    y_pred_combined = lb.transform(list(chain.from_iterable(y_pred)))
        
    tagset = set(lb.classes_) - {'O'}
    tagset = sorted(tagset, key=lambda tag: tag.split('-', 1)[::-1])
    class_indices = {cls: idx for idx, cls in enumerate(lb.classes_)}
    
    return classification_report(
        y_true_combined,
        y_pred_combined,
        labels = [class_indices[cls] for cls in tagset],
        target_names = tagset,
    )

In [4]:
tagger = pycrfsuite.Tagger()
tagger.open('label_scorer.crfsuite')

In [15]:
%%time
y_pred = [tagger.tag(xseq) for xseq in X_test]

CPU times: total: 0 ns
Wall time: 13.1 ms


In [16]:
y_pred

[['comp',
  'subj',
  'case',
  'subj',
  'appos',
  'root',
  'obj',
  'xcomp',
  'obj',
  'case',
  'nmod',
  'nmod',
  'amod'],
 ['subj',
  'advmod',
  'amod',
  'advmod',
  'amod',
  'obj',
  'root',
  'case',
  'det',
  'det',
  'comp',
  'comp',
  'nmod',
  'subj',
  'advmod',
  'amod',
  'ccomp',
  'obj',
  'subj',
  'amod',
  'ccomp',
  'obj'],
 ['xcomp', 'ccomp', 'obj', 'det', 'subj', 'root', 'case', 'det', 'obj'],
 ['nmod', 'subj', 'xcomp', 'root', 'amod', 'obj'],
 ['conj',
  'nmod',
  'ccomp',
  'subj',
  'conj',
  'root',
  'det',
  'comp',
  'obj',
  'xcomp',
  'xcomp',
  'ccomp',
  'obj',
  'iobj',
  'xcomp',
  'obj',
  'conj',
  'det',
  'comp',
  'comp',
  'comp',
  'subj',
  'ccomp',
  'advmod',
  'xcomp',
  'case',
  'obj',
  'nmod'],
 ['xcomp',
  'appos',
  'conj',
  'case',
  'nmod',
  'subj',
  'root',
  'case',
  'obj',
  'comp',
  'iobj',
  'xcomp'],
 ['intj',
  'advmod',
  'root',
  'obj',
  'advmod',
  'nmod',
  'advmod',
  'xcomp',
  'ccomp'],
 ['amod',
  'sub

In [19]:
print(bio_classification_report(y_test, y_pred))

              precision    recall  f1-score   support

      advmod       1.00      1.00      1.00       217
       ammod       1.00      1.00      1.00         2
        amod       1.00      1.00      1.00       136
       appos       1.00      0.97      0.99        35
        case       1.00      1.00      1.00       110
       ccomp       1.00      1.00      1.00       116
        comp       1.00      1.00      1.00        91
        conj       1.00      1.00      1.00        90
         det       1.00      1.00      1.00       100
        intj       1.00      1.00      1.00        31
        iobj       1.00      1.00      1.00        37
        nmod       1.00      1.00      1.00       152
         obj       1.00      1.00      1.00       203
        root       1.00      1.00      1.00       108
        subj       1.00      1.00      1.00       172
       xcomp       1.00      1.00      1.00       121

   micro avg       1.00      1.00      1.00      1721
   macro avg       1.00   

In [20]:
def data_embedding_funct(word):
    final_word_embedding = pd.read_excel('../final_word_embedding.xlsx')
    final_word_embedding = final_word_embedding.rename(columns={'Unnamed: 0' : 'word'})
    return final_word_embedding.loc[final_word_embedding['word'] == word, 'vektor_embedding'].values

In [21]:
def sentences_to_features(sentence, sentence_embedding):
    features = []

    for index in range(len(sentence)):
        feature = {
            'bias': 1.0,
            'word': sentence[index],
            'word_embedding': sentence_embedding[index],
            'is_first': index == 0,
            'is_last': index == len(sentence) - 1,
            'prev_word': '' if index == 0 else sentence[index-1],
            'next_word': '' if index == len(sentence) - 1 else sentence[index+1]
        }
        features.append(feature)
    return features

In [22]:
def proses_pos(sentence):

    crf = pycrfsuite.Tagger()
    crf.open('../crf.model')

    sentence_embs = []

    sentence_split = sentence.split()

    for word in sentence_split:
        sentence_embs.append(data_embedding_funct(word))

    data_temp = {
        'sentences' : sentence_split,
        'word_embs' : sentence_embs
    }

    df_temp = pd.DataFrame(data_temp)
    df_temp['word_embs'] = df_temp['word_embs'].apply(lambda x: x.tolist())

    feature = sentences_to_features(df_temp['sentences'], df_temp['word_embs'])
    return crf.tag(feature)


In [23]:
def make_feature2predict(sentence):
    feature = []
    pos = proses_pos(sentence)
    for index, word in enumerate(sentence.split()):
        feature.append([word, pos[index]])
    return feature

In [24]:
sentence = "Buine tambahe ane besikan bantangne tancebanga ka tanahe ane ngurugin semere muah ane lenan pejanga di sisin semere suud keto lantas kalahina mengkeb"
feature = make_feature2predict(sentence)

In [43]:
input_dim = 200
hidden_dim = 128
model = EdgeScorer(input_dim, hidden_dim)
model.load_state_dict(torch.load('edge_scorer.pth'))
model.eval()

input = edge_score_map(model, sentence)

C:\Users\widi arsa\AppData\Local\Temp\ipykernel_15084\4132443624.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('edge_scorer.pth'))


FileNotFoundError: [Errno 2] No such file or directory: 'edge_scorer.pth'

In [357]:
head_features = construct_head(input, sentence.split())
head_features   

[['Buine', 1],
 ['tambahe', 11],
 ['ane', 1],
 ['besikan', 1],
 ['bantangne', 1],
 ['tancebanga', 1],
 ['ka', 1],
 ['tanahe', 1],
 ['ane', 1],
 ['ngurugin', 1],
 ['semere', 1],
 ['muah', 0],
 ['ane', 1],
 ['lenan', 1],
 ['pejanga', 1],
 ['di', 1],
 ['sisin', 1],
 ['semere', 1],
 ['suud', 1],
 ['keto', 1],
 ['lantas', 1],
 ['kalahina', 1],
 ['mengkeb', 1]]

In [340]:
def feature_construct(feature, head_features):
    features = feature
    for index, value in enumerate(features):
        feature[index].append(head_features[index][1])
    return features

In [358]:
output_label = tagger.tag(sent2features(feature_construct(feature, head_features)))
sentence_list = sentence.split()
map_output = list(zip(output_label, sentence_list))
map_output

[('nmod', 'Buine'),
 ('subj', 'tambahe'),
 ('conj', 'ane'),
 ('subj', 'besikan'),
 ('obj', 'bantangne'),
 ('iobj', 'tancebanga'),
 ('obj', 'ka'),
 ('nmod', 'tanahe'),
 ('conj', 'ane'),
 ('nmod', 'ngurugin'),
 ('subj', 'semere'),
 ('root', 'muah'),
 ('conj', 'ane'),
 ('obj', 'lenan'),
 ('iobj', 'pejanga'),
 ('obj', 'di'),
 ('nmod', 'sisin'),
 ('nmod', 'semere'),
 ('nmod', 'suud'),
 ('subj', 'keto'),
 ('advmod', 'lantas'),
 ('ccomp', 'kalahina'),
 ('obj', 'mengkeb')]

In [1]:
import spacy
from spacy import displacy

# Muat model bahasa
nlp = spacy.load("en_core_web_sm")

# Teks input
text = "SpaCy is an amazing library for NLP tasks."

# Proses teks
doc = nlp(text)

# Visualisasi dependensi
displacy.render(doc, style="dep", jupyter=True)
